In [38]:
#spacy
import spacy
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.tokens import Doc

#gensim
import gensim
from gensim import corpora

#Visualization
from spacy import displacy
import pyLDAvis.gensim_models
from wordcloud import WordCloud
import plotly.express as px
import matplotlib.pyplot as plt

#Data loading/ Data manipulation
import pandas as pd
import numpy as np
import jsonlines

#nltk
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download(['stopwords','wordnet'])

#warning
import warnings 
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Avi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Avi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [39]:
df = pd.read_csv("Resume.csv")
df = df.reindex(np.random.permutation(df.index))
data = df.copy().iloc[
    0:200,
]
data.head()

,ID,Resume_str,Resume_html,Category
2262,39142536,ACCOUNT EXECUTIVE Profile In...,"<div class=""fontsize fontface vmargins hmargin...",BANKING
577,37521676,BUSINESS DEVELOPMENT MANAGER ...,"<div class=""MPR skn-cbg1 fontsize fontface vma...",BUSINESS-DEVELOPMENT
1476,86549455,FINANCE Summary Finance...,"<div class=""fontsize fontface vmargins hmargin...",FINANCE
393,58708773,SUBSTITUTE TEACHER Skills ...,"<div class=""fontsize fontface vmargins hmargin...",TEACHER
2394,14790629,SENIOR MANAGER OF CUSTOMER SUPPORT ...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION


In [40]:
nlp = spacy.load("en_core_web_lg")
#ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk("level_skill.jsonl")
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [41]:
#ruler = nlp.add_pipe("entity_ruler")
#ruler.from_disk(skill_pattern_path)
#nlp.pipe_names

In [42]:
def get_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            subset.append(ent.text)
    myset.append(subset)
    return subset


def unique_skills(x):
    return list(set(x))

In [43]:
clean = []
for i in range(data.shape[0]):
    review = re.sub(
        '(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"',
        " ",
        data["Resume_str"].iloc[i],
    )
    review = review.lower()
    review = review.split()
    lm = WordNetLemmatizer()
    review = [
        lm.lemmatize(word)
        for word in review
        if not word in set(stopwords.words("english"))
    ]
    review = " ".join(review)
    clean.append(review)

In [44]:
data["Clean_Resume"] = clean
data["skills"] = data["Clean_Resume"].str.lower().apply(get_skills)
data["skills"] = data["skills"].apply(unique_skills)
data.head()

,ID,Resume_str,Resume_html,Category,Clean_Resume,skills
2262,39142536,ACCOUNT EXECUTIVE Profile In...,"<div class=""fontsize fontface vmargins hmargin...",BANKING,account executive profile innovative senior ac...,[]
577,37521676,BUSINESS DEVELOPMENT MANAGER ...,"<div class=""MPR skn-cbg1 fontsize fontface vma...",BUSINESS-DEVELOPMENT,business development manager professional summ...,[]
1476,86549455,FINANCE Summary Finance...,"<div class=""fontsize fontface vmargins hmargin...",FINANCE,finance summary finance focal driven improve m...,[]
393,58708773,SUBSTITUTE TEACHER Skills ...,"<div class=""fontsize fontface vmargins hmargin...",TEACHER,substitute teacher skill computer literate dri...,[]
2394,14790629,SENIOR MANAGER OF CUSTOMER SUPPORT ...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION,senior manager customer support summary charis...,[]


In [45]:
fig = px.histogram(
    data, x="Category", title="Distribution of Jobs Categories"
).update_xaxes(categoryorder="total descending")
fig.show()

In [46]:
Job_cat = data["Category"].unique()
Job_cat = np.append(Job_cat, "ALL")

In [47]:
Job_Category = ['BUSINESS-DEVELOPMENT','AVIATION','CONSTRUCTION','DESIGNER','ADVOCATE','TEACHER','ARTS','FITNESS','HR','FINANCE','HEALTHCARE','CONSULTANT'
               ,'SALES','CHEF','DEGITAL-MEDIA','ACCUNTANT','AUTOMOBILE','APPAREL','INFORMATION-TECHNOLOGY',
               'BPO','ENGINEERING','BANKING','AGRIECULTURE','PUBLIC-RELATIONS']

In [48]:
sent = nlp(data["Resume_str"].iloc[0])
displacy.render(sent, style="ent", jupyter=True)

In [49]:
displacy.render(sent[0:10], style="dep", jupyter=True, options={"distance": 90})

In [50]:
patterns = df.Category.unique()
for a in patterns:
    ruler.add_patterns([{"label": "Job-Category", "pattern": a}])

In [51]:
input_resume = "I am Avijit Jana. Functional knowledge of OOPs JAVA,PYTHON,EDA,FLASK API.Knowledge on different Machine Learning technique like Supervised, Unsupervised, Ensemble Technique.Working Experience & Extensive knowledge in python with libraries such as Numpy, Pandas, Matplotlib, Seaborn, Sklearn.Seeking a challenging position in a reputed organization where I can learn new skills, expand my knowledge and leverage my learnings.To get an opportunity where I can make the best of my potential and contribute to the organization’s growth.A highly motivated and hardworking individual looking for a responsible role in a reputable organization.Fluent communication skills and good team working skill.Java, Python,MySQL,MongoDB,WPM,Data Analysis,Python/R, Power BI.Data Cleaning, Data Preparation, Exploratory Data Analysis,MSC IN IT (DATA SCIENCE)"

In [52]:
sent2 = nlp(input_resume)
displacy.render(sent2, style="ent", jupyter=True)